## GRIP:THE SPARKS FOUNDATION

### COMPUTER VISION AND IOT   INTERN

### AUTHOR: DAVID SANDEEP

#### TASK 1: To Implement an object detector which identifies the classes of the objects in an image or video.

In [7]:
import cv2
import numpy as np
import os

# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  

# dnn is deep neural network

classes = []   

with open("coco.names", "r") as f:      

    classes = [line.strip() for line in f.readlines()]  

# Loaded classes from coco file

In [8]:


layer_names = net.getLayerNames()

output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# with output layer we get detection of objects.

colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [9]:
def detecter(image,image_name):
    # Loading image
    img = cv2.imread(image)   

    height, width, channels = img.shape

    # Detecting objects

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)  

    # True refers to converting into rgb format since opencv uses bgr.

    net.setInput(blob)  

    # Passing blob image to yolo algo in network

    outs = net.forward(output_layers)  

    # Giving network to output layer for final result.
    # Showing informations on the screen

    class_ids = []

    confidences = []

    boxes = []

    for out in outs:

        for detection in out:

            scores = detection[5:]

            class_id = np.argmax(scores)

            confidence = scores[class_id]

            if confidence > 0.5:

                # Object detected

                center_x = int(detection[0] * width)

                center_y = int(detection[1] * height)

                w = int(detection[2] * width)

                h = int(detection[3] * height)

                # Rectangle coordinates

                x = int(center_x - w / 2)

                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])

                confidences.append(float(confidence))

                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) 

    # NMS - non max supression

    #print(indexes)

    font = cv2.FONT_HERSHEY_PLAIN

    for i in range(len(boxes)):

        if i in indexes:

            x, y, w, h = boxes[i]

            label = str(classes[class_ids[i]])

            color = colors[i]

            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) 

            # Draw rectangle around boxes. '2' is the width of box.

            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

            # Text in Box to label the object
    cv2.imshow("Image", cv2.resize(img, (960, 540))) 
    cv2.imwrite(image_name, img)
    cv2.waitKey(0) 

     # waitkey stops the output

    cv2.destroyAllWindows()

In [10]:
#using detector function on images
IMAGE_DIRECTORY='images'
for image_name in os.listdir(IMAGE_DIRECTORY):
    image=os.path.join(IMAGE_DIRECTORY, image_name)
    detecter(image,image_name)
    